# Inargural Project 2023

Indholdsfortegnelse


In [2]:
%load_ext autoreload
%autoreload 2

import HouseholdSpecializationModel as HSM
import numpy as np

In [3]:
# Assignment 1

# assign class from module
HSMC = HSM.HouseholdSpecializationModelClass()

# this is how we change the par(ameter) values in HSMC:
HSMC.par.alpha = 0.5
HSMC.par.sigma = 1

# our discrete solution is an SimpleNamespace
solution = HSMC.solve_discrete()

# which we can print as defined in class
HSMC.solve_discrete(do_print=True)

# we can unpack our solution as a dictionary 
dict_sol = vars(solution)


# ... and store as variables
LM, HM, LF, HF = dict_sol.values()
print(LM, HM, LF, HF)


# KJT: missing plotting


# note: i have added a __str__ to the class, so we always can see the baseline parameters
HSMC.par.rho=69
print(HSMC)

HSMC.par.rho=2
print(HSMC)  



/Users/konradthide/Desktop/01.06_IntroProg/assignments/projects-2023-scb/inauguralproject/HouseholdSpecializationModel.py:94: RuntimeWarning: invalid value encountered in reciprocal
  utility = np.fmax(Q,1e-8)**(1-par.rho)*(1-par.rho)**-1


start solve_discrete()
LM = 4.5000
HM = 4.5000
LF = 4.5000
HF = 4.5000
end solve_discrete()
4.5 4.5 4.5 4.5

 The baseline parameters in the model is: 
1. Preferences           = (rho, epsilon, omega, nu) = (69.00, 1.00, 0.50, 0.001) 
2. Household production  = (alpha, sigma )           = (0.50, 1.00) 
3. Wages                 = (wM,wF)                   = (1.00, 1.00) 


 The baseline parameters in the model is: 
1. Preferences           = (rho, epsilon, omega, nu) = (2.00, 1.00, 0.50, 0.001) 
2. Household production  = (alpha, sigma )           = (0.50, 1.00) 
3. Wages                 = (wM,wF)                   = (1.00, 1.00) 



In [4]:
# assignment 3

print(HSMC)

# solve continously
HSMC.solve(do_print=True)

# results seems to be aligned with the discrete solution, check for base values
HSMC.solve_discrete(do_print=True)

# for different values for wF, see function .solve_wF_vec, not sure if right way.
wF_vec_SOL = HSMC.solve_wF_vec(do_print=True)

print(HSMC)

# KJT: need to plot.



 The baseline parameters in the model is: 
1. Preferences           = (rho, epsilon, omega, nu) = (2.00, 1.00, 0.50, 0.001) 
2. Household production  = (alpha, sigma )           = (0.50, 1.00) 
3. Wages                 = (wM,wF)                   = (1.00, 1.00) 

start solve()
LM = 4.4540
HM = 4.4540
LF = 4.4540
HF = 4.4540
end solve()
start solve_discrete()
LM = 4.5000
HM = 4.5000
LF = 4.5000
HF = 4.5000
end solve_discrete()
start solve_wF_vec()
wf_vec = [0.8 0.9 1.  1.1 1.2]
LM_vec = [5.90219912 4.67971036 4.45400058 4.24673723 4.05670539]
HM_vec = [4.08706788 4.48915532 4.45400056 4.41920581 4.385003  ]
LF_vec = [2.87942497 4.298631   4.45400058 4.5844395  4.69404487]
HF_vec = [5.09573769 4.4935318  4.45400058 4.41534835 4.3777336 ]
end solve_wF_vec()

 The baseline parameters in the model is: 
1. Preferences           = (rho, epsilon, omega, nu) = (2.00, 1.00, 0.50, 0.001) 
2. Household production  = (alpha, sigma )           = (0.50, 1.00) 
3. Wages                 = (wM,wF)     

## UNDERSTAND MODULE

In [19]:
# for solve_discrete

x = np.linspace(0,24,49)
LM,HM,LF,HF = np.meshgrid(x,x,x,x) 

LM = LM.ravel()

print(LM, len(LM))
print(49**4)

[ 0.  0.  0. ... 24. 24. 24.] 5764801
5764801
